In [3]:
# =========================
# STEP 1 — ENV + IMPORTS (FINALIZED)
# =========================

# ---- 1) Install dependencies ----
try:
    import langchain, chromadb, gradio
except Exception:
    import sys, subprocess
    pkgs = [
        "langchain>=0.2.0",
        "langchain-core>=0.2.0",
        "langchain-community>=0.2.0",
        "langchain-openai>=0.1.0",
        "langchain-text-splitters>=0.2.0",
        "chromadb>=0.5.0",
        "pypdf>=4.0.0",
        "tiktoken>=0.7.0",
        "gradio>=4.0.0",
        "openai>=1.40.0"
    ]
    subprocess.check_call([sys.executable, "-m", "pip", "install", *pkgs])

# ---- 2) API key setup ----
import os
from getpass import getpass

if not os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass("Enter your OPENAI_API_KEY: ")

# ---- 3) Imports ----
from openai import OpenAI
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
import gradio as gr
import tiktoken, chromadb, pathlib, sys, platform

# ---- 4) Constants ----
# ✅ Use a writable directory (avoid permission issues)
DATA_DIR = pathlib.Path("/mnt/data") if pathlib.Path("/mnt/data").exists() else pathlib.Path.cwd()
PDF_PATH = DATA_DIR / "the_nestle_hr_policy_pdf_2012.pdf"
EMBEDDING_MODEL = "text-embedding-3-small"
CHAT_MODEL = "gpt-3.5-turbo"

# ---- 5) Sanity checks ----
client = OpenAI()
print("✅ Environment ready.")
print(f"Python: {sys.version.split()[0]}  |  Platform: {platform.system()} {platform.release()}")
print(f"LangChain: {langchain.__version__}  |  ChromaDB: {chromadb.__version__}  |  Gradio: {gr.__version__}")
print(f"PDF present: {PDF_PATH.exists()}  -> {PDF_PATH}")

# Tokenizer test (verifies tiktoken works)
enc = tiktoken.get_encoding("cl100k_base")
print("Token test (hello world):", len(enc.encode("hello world")))


✅ Environment ready.
Python: 3.10.2  |  Platform: Linux 5.15.0-1064-aws
LangChain: 1.0.3  |  ChromaDB: 1.3.2  |  Gradio: 4.7.1
PDF present: True  -> /voc/work/the_nestle_hr_policy_pdf_2012.pdf
Token test (hello world): 2


In [5]:
!pip install pysqlite3-binary
# ---- Patch SQLite if too old ----
try:
    import sqlite3
    import pysqlite3
    import sys
    sys.modules["sqlite3"] = sys.modules.pop("pysqlite3")
    print("✅ Patched sqlite3 with pysqlite3 backend.")
except Exception as e:
    print("⚠️ Could not patch sqlite3, fallback to system version:", e)


Defaulting to user installation because normal site-packages is not writeable
✅ Patched sqlite3 with pysqlite3 backend.


In [4]:
#!pip install --upgrade --force-reinstall "gradio==4.29.0" 


Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 25.0 MB/s  0:00:00 eta 0:00:01


In [4]:
# =========================
# STEP 2 — LOAD & SPLIT PDF
# =========================

from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import pickle

# ---- 1) Load the PDF ----
print("📄 Loading HR Policy PDF...")
loader = PyPDFLoader(str(PDF_PATH))
documents = loader.load()   # list of Document objects (one per page)
print(f"✅ Loaded {len(documents)} pages from: {PDF_PATH.name}")

# ---- 2) Inspect first page (optional) ----
print("\n--- Sample text snippet ---")
print(documents[0].page_content[:400], "\n")

# ---- 3) Split text into smaller chunks ----
#   Small chunks improve embedding relevance.
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,       # about 200–250 tokens
    chunk_overlap=200,     # keep context continuity
    separators=["\n\n", "\n", ".", " "],
)

chunks = splitter.split_documents(documents)
print(f"✅ Split into {len(chunks)} chunks (~{len(chunks[0].page_content)} chars each)")

# ---- 4) Preview a few chunks (optional) ----
for i, c in enumerate(chunks[:2]):
    print(f"\n🧩 Chunk {i+1}:\n{c.page_content[:300]}...")

# ---- 5) (Optional) Save chunks for reuse later ----
CHUNKS_PATH = DATA_DIR / "hr_policy_chunks.pkl"
with open(CHUNKS_PATH, "wb") as f:
    pickle.dump(chunks, f)

print(f"\n💾 Chunks saved to: {CHUNKS_PATH}")


📄 Loading HR Policy PDF...
✅ Loaded 8 pages from: the_nestle_hr_policy_pdf_2012.pdf

--- Sample text snippet ---
Policy
Mandatory
September  2012
The Nestlé  
Human Resources Policy 

✅ Split into 20 chunks (~68 chars each)

🧩 Chunk 1:
Policy
Mandatory
September  2012
The Nestlé  
Human Resources Policy...

🧩 Chunk 2:
Policy
Mandatory
September 
 20
12
Issuing departement
Hum
an Resources
Target audience 
All
 employees
Approver
Executive Board, Nestlé S.A.
Repository
All Nestlé Principles and Policies, Standards and  
Guidelines can be found in the Centre online repository at:  
http://intranet.nestle.com/nestle...

💾 Chunks saved to: /voc/work/hr_policy_chunks.pkl


In [2]:
!pip install pysqlite3-binary
# ---- Patch SQLite if too old ----
try:
    import sqlite3
    import pysqlite3
    import sys
    sys.modules["sqlite3"] = sys.modules.pop("pysqlite3")
    print("✅ Patched sqlite3 with pysqlite3 backend.")
except Exception as e:
    print("⚠️ Could not patch sqlite3, fallback to system version:", e)

Defaulting to user installation because normal site-packages is not writeable
✅ Patched sqlite3 with pysqlite3 backend.


In [7]:
# ===============================
# STEP 3 — EMBEDDINGS + IN-MEMORY VECTOR DB (Final Fixed)
# ===============================

from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
import chromadb

print("🔢 Creating text embeddings using OpenAI (in-memory mode)...")

# Defining chunk path
import pickle, pathlib
CHUNKS_PATH = pathlib.Path("hr_policy_chunks.pkl")
with open(CHUNKS_PATH, "rb") as f:
    chunks = pickle.load(f)
print(f"✅ Loaded {len(chunks)} chunks from disk")


# 1️⃣ Create embeddings
embedding_function = OpenAIEmbeddings(model="text-embedding-3-small")

# 2️⃣ Use in-memory Chroma client (no disk writes)
client = chromadb.Client()
print("✅ Using in-memory Chroma client (no file-system writes).")

# 3️⃣ Build vector store
vector_store = Chroma.from_documents(
    documents=chunks,
    embedding=embedding_function,
    client=client,
    collection_name="hr_policy_memory"
)
print("✅ Vector store built completely in memory.")

# 4️⃣ Test retrieval with the new API
sample_query = "What is the policy on employee training and development?"
retriever = vector_store.as_retriever(search_kwargs={"k": 2})
results = retriever.invoke(sample_query)  # <-- modern method

print("\n🔍 Retrieval test results:")
for i, doc in enumerate(results, start=1):
    print(f"\nResult {i}:\n{doc.page_content[:400]}...")


Failed to send telemetry event ClientStartEvent: module 'chromadb' has no attribute 'get_settings'
Failed to send telemetry event ClientCreateCollectionEvent: module 'chromadb' has no attribute 'get_settings'


🔢 Creating text embeddings using OpenAI (in-memory mode)...
✅ Loaded 20 chunks from disk
✅ Using in-memory Chroma client (no file-system writes).
✅ Vector store built completely in memory.


Failed to send telemetry event CollectionQueryEvent: module 'chromadb' has no attribute 'get_settings'



🔍 Retrieval test results:

Result 1:
The Nestlé Human Resources Policy
4
Learning is part of the Company culture.
Employees at all levels are systematically 
encouraged to consider how they upgrade their 
knowledge and skills.
The Company determines training and deve-
lopment priorities. The responsibility for turning 
these into actions is shared between employees, 
line managers and the Human Resources. 
Experience and on-the-job t...

Result 2:
teams are encouraged to acquire additional skills, 
enrich job content and widen accountability.
Nestlé also offers a comprehensive range of 
training activities and methodologies to support 
everyone’s learning and growth. Attending 
a programme should never be considered 
as a reward but as a component of on-going 
development.
Additionally, corporate leadership programmes 
help us develop and r...


In [8]:
# ============================
# STEP 4 — QUESTION–ANSWER CHAIN
# ============================

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# 1️⃣  Initialize GPT model
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.2)

# 2️⃣  Define a clear prompt template
prompt = ChatPromptTemplate.from_template("""
You are an AI-powered HR Assistant.
Answer the question strictly using the context provided below.

Context:
{context}

Question:
{question}

If the answer is not contained in the context, respond:
"I couldn’t find this information in the HR policy document."
""")

# 3️⃣  Build the retrieval + generation pipeline
retriever = vector_store.as_retriever(search_kwargs={"k": 3})

# Combine:  user question → retrieve → format → LLM → output
qa_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# 4️⃣  Test the pipeline
query = "What does the HR policy say about employee relations?"
response = qa_chain.invoke(query)

print("🔍 Question:", query)
print("💬 Answer:", response)


🔍 Question: What does the HR policy say about employee relations?
💬 Answer: The HR policy emphasizes the importance of fair and constructive dialogues between the company and employee representatives, overcoming difficulties, reaching sustainable agreements, and implementing them. It also mentions the commitment to establishing flat and flexible structures with minimal levels of management and broad spans of control to enable people development, increase efficiency, and ease implementation of the "Nestlé Management and Leadership Principles."


In [9]:
# ============================
# STEP 5 — GRADIO CHATBOT UI
# ============================

import gradio as gr

# --- 1️⃣  Chatbot function ---
def hr_chatbot(user_query):
    """
    Receives a user's question, routes it through the retrieval+LLM chain,
    and returns the answer.
    """
    try:
        result = qa_chain.invoke(user_query)
        return result
    except Exception as e:
        return f"⚠️ Error: {str(e)}"


# --- 2️⃣  Gradio interface layout ---
chat_ui = gr.Interface(
    fn=hr_chatbot,
    inputs=gr.Textbox(
        lines=2,
        placeholder="Ask a question about the HR policy...",
        label="💬 Your Question"
    ),
    outputs=gr.Textbox(
        label="🤖 HR Assistant Answer"
    ),
    title="AI-Powered HR Assistant",
    description=(
        "Ask any question related to the HR policy document. "
        "The assistant uses OpenAI GPT-3.5-Turbo with document-aware retrieval."
    ),
    examples=[
        ["What does the policy say about employee training?"],
        ["Explain the company’s stance on work-life balance."],
        ["How are performance evaluations conducted?"]
    ],
)

# --- 3️⃣  Launch the web interface ---
chat_ui.launch(share=True)     # gives a public link
# OR, for Jupyter inline display
chat_ui.launch(inline=True)

#chat_ui.launch(debug=True, share=False)


IMPORTANT: You are using gradio version 4.29.0, however version 4.44.1 is available, please upgrade.
--------
Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://bba31dc3dcb3354df4.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----

To create a public link, set `share=True` in `launch()`.


In [18]:
#!pip install --upgrade --force-reinstall "gradio==4.29.0" 


Defaulting to user installation because normal site-packages is not writeable
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached orjson-3.11.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (41 kB)
  Using cached pydantic-2.12.3-py3-none-any.whl.metadata (87 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached pydantic_core-2.41.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.3 kB)
  Using cached typing_inspection-0.4.2-py3-none-any.whl.metadata (2.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 47.6 MB/s  0:00:006m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.2/731.2 kB 5.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 48.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 73.3 MB/s  0:00:006m0:00:01
Using cached orjson-3.11.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (136 kB)
   ━━━━━━━━

In [2]:
!pip install --upgrade --force-reinstall "huggingface_hub==0.24.5"


Defaulting to user installation because normal site-packages is not writeable
  Using cached filelock-3.20.0-py3-none-any.whl.metadata (2.1 kB)
  Using cached fsspec-2025.10.0-py3-none-any.whl.metadata (10 kB)
  Using cached packaging-25.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached pyyaml-6.0.3-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl.metadata (2.4 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached idna-3.11-py3-none-any.whl.metadata (8.4 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2025.10.5-py3-none-any.whl.metadata (2.5 kB)
Using cached fsspec-2025.10.0-py3-none-any.whl (200 kB)
Using cached packaging-25.0-py3-none-any.whl (66 kB)
Using cached pyyaml-6.0.3-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28

In [10]:
# ============================
# STEP 6 — TEST & EVALUATE CHATBOT
# ============================

import time

# 1️⃣  Define a few realistic HR questions
test_questions = [
    "What does the HR policy say about employee training?",
    "How does the company ensure work-life balance?",
    "Explain the policy on performance evaluation.",
    "What is the approach towards diversity and inclusion?",
    "Who is responsible for employee safety?"
]

# 2️⃣  Run quick evaluation
print("🤖  Evaluating AI-Powered HR Assistant...\n")
for q in test_questions:
    start = time.time()
    try:
        ans = qa_chain.invoke(q)
        print(f"Q: {q}\nA: {ans}\n⏱  {time.time()-start:.2f}s\n{'-'*70}")
    except Exception as e:
        print(f"⚠️  Error on question: {q}\n{e}\n{'-'*70}")


🤖  Evaluating AI-Powered HR Assistant...

Q: What does the HR policy say about employee training?
A: The HR policy emphasizes that learning is part of the company culture, and employees are encouraged to upgrade their knowledge and skills. The company determines training and development priorities, with responsibility shared between employees, line managers, and HR. Experience and on-the-job training are primary sources of learning, and managers are responsible for guiding and coaching employees to succeed in their current positions. Continuous improvement, sharing knowledge, and participating in training activities are also encouraged.
⏱  3.16s
----------------------------------------------------------------------
Q: How does the company ensure work-life balance?
A: The company ensures work-life balance by providing flexible working conditions whenever possible, encouraging employees to have outside interests, especially community involvement, and supporting a better balance of privat

In [16]:
# ======================================================
# STEP 6 — FINAL PROJECT BLOCK (Manual Memory + Logging + Tests)
# ======================================================

import csv, datetime, time
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
import gradio as gr

# 1️⃣ Simple manual memory (list-based)
conversation_history = []  # stores tuples of (question, answer)

def get_chat_history_text():
    if not conversation_history:
        return "No previous conversation."
    hist = ""
    for q, a in conversation_history[-5:]:  # keep last 5 turns
        hist += f"User: {q}\nAssistant: {a}\n"
    return hist

# 2️⃣ Prompt template including previous chat
prompt_with_history = ChatPromptTemplate.from_template("""
You are an AI-powered HR Assistant.
Use ONLY the HR policy context below to answer accurately and concisely.
Keep prior conversation in mind for continuity.

Previous conversation:
{chat_history}

Context:
{context}

Question:
{question}

If the answer is not contained in the context, respond:
"I couldn’t find this information in the HR policy document."
""")

# 3️⃣ Build full chain
qa_with_history = (
    {
        "context": retriever,
        "question": RunnablePassthrough(),
        "chat_history": lambda _: get_chat_history_text(),
    }
    | prompt_with_history
    | llm
    | StrOutputParser()
)

print("✅ HR Assistant with manual memory ready.\n")

# 4️⃣ CSV logging
LOG_FILE = "chat_log.csv"
def log_interaction(question, answer):
    with open(LOG_FILE, "a", newline="", encoding="utf-8") as f:
        csv.writer(f).writerow([datetime.datetime.now(), question, answer])

# 5️⃣ Chatbot function
def hr_chatbot(user_query):
    try:
        result = qa_with_history.invoke(user_query)
        conversation_history.append((user_query, result))
        log_interaction(user_query, result)
        return result
    except Exception as e:
        return f"⚠️ Error: {e}"

# 6️⃣ Gradio interface
chat_ui = gr.Interface(
    fn=hr_chatbot,
    inputs=gr.Textbox(lines=2, placeholder="Ask about HR policy..."),
    outputs=gr.Textbox(label="🤖 HR Assistant Answer"),
    title="AI-Powered HR Assistant with Memory",
    description="Chat with an HR Assistant powered by OpenAI GPT-3.5 and your HR policy document.",
    examples=[
        ["What does the policy say about employee training?"],
        ["Who ensures workplace safety?"],
        ["Explain the policy on performance evaluation."]
    ],
)

chat_ui.launch(share=True)


✅ HR Assistant with manual memory ready.

IMPORTANT: You are using gradio version 4.29.0, however version 4.44.1 is available, please upgrade.
--------
Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://d0bfa38b4d76c3b7fb.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
